# Propositionalization: Predicting air pollution in Beijing

In this notebook we will compare getML to featuretools and tsfresh, both of which open-source libraries for feature engineering. We find that advanced algorithms featured in getML yield significantly better predictions on this dataset. We then discuss why that is.

Summary:

- Prediction type: __Regression model__
- Domain: __Air pollution__
- Prediction target: __pm 2.5 concentration__ 
- Source data: __Multivariate time series__
- Population size: __41757__

## Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

As our example dataset, we use a publicly available dataset on air pollution in Beijing, China (https://archive.ics.uci.edu/dataset/381/beijing+pm2+5+data). For further details about the data set refer to [the full notebook](../air_pollution.ipynb).

## Analysis

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)
4. [Conclusion](#4.-Conclusion)

In [1]:
import datetime
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
from pathlib import Path

import sys
import time
from urllib import request

import getml
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

%matplotlib inline

In [2]:
parent = Path(os.getcwd()).parent.as_posix()

if parent not in sys.path:
   sys.path.append(parent) 

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

### 1. Loading data

#### 1.1 Download from source

We begin by downloading the data from the UCI Machine Learning repository.

In [3]:
FEATURETOOLS_FILES = ["featuretools_training.csv", "featuretools_test.csv"]

for fname in FEATURETOOLS_FILES:
    if not os.path.exists(fname):
        fname, res = request.urlretrieve(
            "https://static.getml.com/datasets/air_pollution/featuretools/" + fname,
            fname,
        )

In [4]:
TSFRESH_FILES = ["tsfresh_training.csv", "tsfresh_test.csv"]

for fname in TSFRESH_FILES:
    if not os.path.exists(fname):
        fname, res = request.urlretrieve(
            "https://static.getml.com/datasets/air_pollution/tsfresh/" + fname, fname
        )

In [5]:
fname = "PRSA_data_2010.1.1-2014.12.31.csv"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/" + fname,
        fname,
    )

#### 1.2 Prepare data for tsfresh and getML

Our our goal is to predict the pm2.5 concentration from factors such as weather or time of day. However, there are some **missing entries** for pm2.5, so we get rid of them.

In [6]:
data_full_pandas = pd.read_csv(fname)

data_full_pandas = data_full_pandas[
    data_full_pandas["pm2.5"] == data_full_pandas["pm2.5"]
]

tsfresh requires a date column, so we build one.

In [7]:
def add_leading_zero(val):
    if len(str(val)) == 1:
        return "0" + str(val)
    return str(val)


data_full_pandas["month"] = [add_leading_zero(val) for val in data_full_pandas["month"]]

data_full_pandas["day"] = [add_leading_zero(val) for val in data_full_pandas["day"]]

data_full_pandas["hour"] = [add_leading_zero(val) for val in data_full_pandas["hour"]]


def make_date(year, month, day, hour):
    return year + "-" + month + "-" + day + " " + hour + ":00:00"


data_full_pandas["date"] = [
    make_date(str(year), month, day, hour)
    for year, month, day, hour in zip(
        data_full_pandas["year"],
        data_full_pandas["month"],
        data_full_pandas["day"],
        data_full_pandas["hour"],
    )
]

tsfresh also requires the time series to have ids. Since there is only a single time series, that series has the same id.

In [8]:
data_full_pandas["id"] = 1

The dataset now contains many columns that we do not need or that tsfresh cannot process. For instance, *cbwd* might actually contain useful information, but it is a categorical variable, which is difficult to handle for tsfresh, so we remove it.

We also want to split our data into a training and testing set.

In [9]:
data_train_pandas = data_full_pandas[data_full_pandas["year"] < 2014]
data_test_pandas = data_full_pandas[data_full_pandas["year"] == 2014]
data_full_pandas = data_full_pandas

In [10]:
def remove_unwanted_columns(df):
    del df["cbwd"]
    del df["year"]
    del df["month"]
    del df["day"]
    del df["hour"]
    del df["No"]
    return df


data_full_pandas = remove_unwanted_columns(data_full_pandas)
data_train_pandas = remove_unwanted_columns(data_train_pandas)
data_test_pandas = remove_unwanted_columns(data_test_pandas)

In [11]:
data_full_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
43819,8.0,-23,-2.0,1034.0,231.97,0,0,2014-12-31 19:00:00,1
43820,10.0,-22,-3.0,1034.0,237.78,0,0,2014-12-31 20:00:00,1
43821,10.0,-22,-3.0,1034.0,242.70,0,0,2014-12-31 21:00:00,1
43822,8.0,-22,-4.0,1034.0,246.72,0,0,2014-12-31 22:00:00,1


We then **load the data into the getML engine**. We begin by setting a project.

In [12]:
getml.engine.launch(home_directory=Path.home(), allow_remote_ips=True, token='token')
getml.engine.set_project("air_pollution")

getML engine is already running.
Loading pipelines... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          

Connected to project 'air_pollution'


In [13]:
df_full = getml.data.DataFrame.from_pandas(data_full_pandas, name="full")
df_full["date"] = df_full["date"].as_ts()

We need to **assign roles** to the columns, such as defining the target column.

In [14]:
df_full.set_role(["date"], getml.data.roles.time_stamp)
df_full.set_role(["pm2.5"], getml.data.roles.target)
df_full.set_role(
    ["DEWP", "TEMP", "PRES", "Iws", "Is", "Ir"], getml.data.roles.numerical
)
df_full

name,date,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,id
role,time_stamp,target,numerical,numerical,numerical,numerical,numerical,numerical,unused_float
unit,"time stamp, comparison only",,,,,,,,
0,2010-01-02,129,-16,-4,1020,1.79,0,0,1
1,2010-01-02 01:00:00,148,-15,-4,1020,2.68,0,0,1
2,2010-01-02 02:00:00,159,-11,-5,1021,3.57,0,0,1
3,2010-01-02 03:00:00,181,-7,-5,1022,5.36,1,0,1
4,2010-01-02 04:00:00,138,-7,-5,1022,6.25,2,0,1
,...,...,...,...,...,...,...,...,...
41752,2014-12-31 19:00:00,8,-23,-2,1034,231.97,0,0,1
41753,2014-12-31 20:00:00,10,-22,-3,1034,237.78,0,0,1


In [15]:
split = getml.data.split.time(
    population=df_full, time_stamp="date", test=getml.data.time.datetime(2014, 1, 1)
)
split

,
0,train
1,train
2,train
3,train
4,train
,...


### 2. Predictive modeling

#### 2.1 Propositionalization with getML's FastProp

In [16]:
time_series = getml.data.TimeSeries(
    population=df_full,
    alias="population",
    split=split,
    time_stamps="date",
    memory=getml.data.time.days(1),
)

time_series

data model

  population:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...

    joins:
    - right: 'full'
      time_stamps: (population.date, full.date)
      relationship: 'many-to-many'
      memory: 86400.0
      lagged_targets: False

  full:
    columns:
    - DEWP: numerical
    - TEMP: numerical
    - PRES: numerical
    - Iws: numerical
    - Is: numerical
    - ...


container

  population
      subset   name    rows   type
  0   test     full    8661   View
  1   train    full   33096   View

  peripheral
      name    rows   type     
  0   full   41757   DataFrame

In [17]:
fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastProp.agg_sets.All,
)

pipe_fp_fl = getml.pipeline.Pipeline(
    tags=["memory: 1d", "simple features"],
    data_model=time_series.data_model,
    feature_learners=[fast_prop],
)

pipe_fp_fl

Pipeline(data_model='population',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=['full'],
         predictors=[],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['memory: 1d', 'simple features'])

In [18]:
pipe_fp_fl.check(time_series.train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:02, remaining: 00:00]          

OK.


In [19]:
benchmark = Benchmark()

In [20]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)
    fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Trying 331 features... 100% |██████████| [elapsed: 00:05, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:5.370066

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:03, remaining: 00:00]          



In [21]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
FastProp: Building features... 100% |██████████| [elapsed: 00:01, remaining: 00:00]          



In [22]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [23]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:16, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:16.267519



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

In [24]:
pipe_fp_pr.score(fastprop_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:13:14,fastprop_train,pm2.5,38.3028,55.2472,0.6438
1,2024-02-21 16:13:14,fastprop_test,pm2.5,44.2526,63.4191,0.5462


#### 2.2 Using featuretools

To make things a bit easier, we have written a high-level wrapper around featuretools which we placed in a separate module (`utils`).

In [25]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(days=0),
    memory=pd.Timedelta(days=1),
    column_id="id",
    time_stamp="date",
    target="pm2.5",
)

In [26]:
with benchmark("featuretools"):
    featuretools_training = ft_builder.fit(data_train_pandas)

featuretools_test = ft_builder.transform(data_test_pandas)

featuretools: Trying features...
Selecting the best out of 298 features...
Time taken: 0h:34m:1.838204



In [27]:
df_featuretools_training = getml.data.DataFrame.from_pandas(
    featuretools_training, name="featuretools_training"
)
df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test"
)

In [28]:
def set_roles_featuretools(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_featuretools_training = set_roles_featuretools(df_featuretools_training)
df_featuretools_test = set_roles_featuretools(df_featuretools_test)

In [29]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["featuretools", "memory: 1d"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d'])

In [30]:
pipe_ft_pr.check(df_featuretools_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


In [31]:
pipe_ft_pr.fit(df_featuretools_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:10, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:10.460135



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['featuretools', 'memory: 1d'])

In [32]:
pipe_ft_pr.score(df_featuretools_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:56:07,featuretools_training,pm2.5,38.277,54.8781,0.6506
1,2024-02-21 16:56:07,featuretools_test,pm2.5,43.9151,62.5672,0.5594


#### 2.3 Using tsfresh

tsfresh is a rather low-level library. To make things a bit easier, we have written a high-level wrapper which we placed in a separate module (`utils`).

To limit the memory consumption, we undertake the following steps:

- We limit ourselves to a memory of 1 day from any point in time. This is necessary, because tsfresh duplicates records for every time stamp. That means that looking back 7 days instead of one day, the memory consumption would be  seven times as high.
- We extract only tsfresh's **MinimalFCParameters** and **IndexBasedFCParameters** (the latter is a superset of **TimeBasedFCParameters**).

In order to make sure that tsfresh's features can be compared to getML's features, we also do the following:

- We apply tsfresh's built-in feature selection algorithm.
- Of the remaining features, we only keep the 40 features most correlated with the target (in terms of the absolute value of the correlation).
- We add the original columns as additional features.


In [33]:
data_train_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
35059,22.0,-19,7.0,1013.0,114.87,0,0,2013-12-31 19:00:00,1
35060,18.0,-21,7.0,1014.0,119.79,0,0,2013-12-31 20:00:00,1
35061,23.0,-21,7.0,1014.0,125.60,0,0,2013-12-31 21:00:00,1
35062,20.0,-21,6.0,1014.0,130.52,0,0,2013-12-31 22:00:00,1


In [34]:
tsfresh_builder = TSFreshBuilder(
    num_features=200, memory=24, column_id="id", time_stamp="date", target="pm2.5"
)

with benchmark("tsfresh"):
    tsfresh_training = tsfresh_builder.fit(data_train_pandas)

tsfresh_test = tsfresh_builder.transform(data_test_pandas)

Feature Extraction: 100%|██████████| 40/40 [00:29<00:00,  1.35it/s]


Selecting the best out of 78 features...
Time taken: 0h:1m:30.572509



Feature Extraction: 100%|██████████| 40/40 [00:07<00:00,  5.61it/s]


tsfresh does not contain built-in machine learning algorithms. In order to ensure a fair comparison, we use the exact same machine learning algorithm we have also used for getML: An XGBoost regressor with all hyperparameters set to their default value.

In order to do so, we load the tsfresh features into the getML engine.

In [35]:
df_tsfresh_training = getml.data.DataFrame.from_pandas(
    tsfresh_training, name="tsfresh_training"
)
df_tsfresh_test = getml.data.DataFrame.from_pandas(tsfresh_test, name="tsfresh_test")

As usual, we need to set roles:

In [36]:
def set_roles_tsfresh(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.roles.unused, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df


df_tsfresh_training = set_roles_tsfresh(df_tsfresh_training)
df_tsfresh_test = set_roles_tsfresh(df_tsfresh_test)

In this case, our pipeline is very simple. It only consists of a single XGBoostRegressor.

In [37]:
predictor = getml.predictors.XGBoostRegressor()

pipe_tsf_pr = getml.pipeline.Pipeline(
    tags=["tsfresh", "memory: 1d"], predictors=[predictor]
)

pipe_tsf_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d'])

In [38]:
pipe_tsf_pr.check(df_tsfresh_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Checking... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.


In [39]:
pipe_tsf_pr.fit(df_tsfresh_training)

Checking data model...
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          

OK.
Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
XGBoost: Training as predictor... 100% |██████████| [elapsed: 00:06, remaining: 00:00]          

Trained pipeline.
Time taken: 0h:0m:5.783919



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function='SquareLoss',
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['tsfresh', 'memory: 1d'])

In [40]:
pipe_tsf_pr.score(df_tsfresh_test)

Staging... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          
Preprocessing... 100% |██████████| [elapsed: 00:00, remaining: 00:00]          



,date time,set used,target,mae,rmse,rsquared
0,2024-02-21 16:58:06,tsfresh_training,pm2.5,40.8917,57.9517,0.6099
1,2024-02-21 16:58:06,tsfresh_test,pm2.5,47.1106,66.6,0.5015


### 3. Comparison

In [41]:
num_features = dict(
    fastprop=289,
    featuretools=114,
    tsfresh=72,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
    benchmark.runtimes["tsfresh"] / num_features["tsfresh"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[
            benchmark.runtimes["fastprop"],
            benchmark.runtimes["featuretools"],
            benchmark.runtimes["tsfresh"],
        ],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
            benchmark.runtimes["tsfresh"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae, pipe_tsf_pr.mae],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse, pipe_tsf_pr.rmse],
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared, pipe_tsf_pr.rsquared],
    )
)

comparison.index = ["getML: FastProp", "featuretools", "tsfresh"]

comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,mae,rmse,rsquared
getML: FastProp,0 days 00:00:08.637513,289,33.458244,1.000000,1.000000,44.252635,63.419113,0.546164
featuretools,0 days 00:34:01.839960,114,0.055832,236.392114,599.266495,43.915071,62.567175,0.559369
tsfresh,0 days 00:01:30.572946,72,0.794939,10.485998,42.089066,47.110594,66.599982,0.501524


In [42]:
# export for further use
comparison.to_csv("comparisons/air_pollution.csv")

### 4. Conclusion

We have compared getML's feature learning algorithms to tsfresh's brute-force feature engineering approaches on a data set related to air pollution in China. We found that getML significantly outperforms featuretools and tsfresh. These results are consistent with the view that feature learning can yield significant improvements over simple propositionalization approaches.

However, there are other datasets on which simple propositionalization performs well. Our suggestion is therefore to think of algorithms like `FastProp` and `RelMT` as tools in a toolbox. If a simple tool like `FastProp` gets the job done, then use that. But when you need more advanced approaches, like `RelMT`, you should have them at your disposal as well.

You are encouraged to reproduce these results.